# Game and Argumentation: Time for a Family Reunion

This notebook aims to demonstrate the paper "Game and Argumentation: Time for a Family Reunion" accepted by XLoKR 2023

Lets start with a move graph

![image](files/move_graph.png)

The move graph above defines a game $G = (V, E)$ with positions $V$ and moves $E$. Based on the $win(X) ← move(X, Y ), ¬ win(Y ).$, we can solve this game by
1) Identifying sink nodes, or lost positions which did not have outgoing moves
2) Iteratively running the following
- Position $x$ is green (won) if $∃ move x → y$ and position $y$ is already red (lost)
- Position $x$ is red (lost) if $∀ move x → y$ and position $y$ is already green (lost)

## Graphs to Facts

To help future reasoning, here we turn the graphs into facts

In [1]:
%%file files/move_graph.dlv

#maxint = 100.

m(a,b).
m(a,c).
m(k,l).
m(c,d).
m(c,e).
m(l,e).
m(l,m).
m(d,e).
m(e,d).
m(e,m).
m(m,e).
m(d,f).
m(d,g).
m(g,d).
m(e,h).
m(m,n).
m(n,m).

Overwriting files/move_graph.dlv


In [2]:
%%file files/game_solve.dlv

% Positions
p(X) :- m(X,_).
p(X) :- m(_,X).

% win_u: underestimate of WON positions
u(S1, X) :-
	m(X,Y),
	not o(S,Y),
	nxt(S,S1). % S1 = S + 1

%: win_o: overestimate of WON positions
o(S, X) :-
	m(X,Y),
	not u(S,Y),
	nxt(S,_).
% GREEN (won) positions 
g(X) :-
	fg(_,X).  

% YELLOW (drawn) positions
y(X) :-
	p(X),
	not g(X),
	not r(X).

% RED (lost) positions
r(X) :- fr(_,X).

% State generation for AFP 
nxt(0,1).
nxt(S,S1) :-			% S1 (=S+1) is a state,
	nxt(_,S),		% ... if S is a state
	chg(S),			% ... which changes
	S1=S+1.    

% change(S)
chg(0).				% in 0 there is change
chg(S) :-			% in S there is change
	fg(S,_).		% ... if there is some FirstGreen

% final(S)
fin(S) :-			% S is the final state
	nxt(_,S),     
	not chg(S).		% ... if there is no change in S

% FirstGreen(State, Position)
fg(S1,X) :- 		       % position X is first green in S1 (=S+1)
	nxt(S,S1),
	u(S1,X),               % ... if win_u(S1,X)
	not u(S,X).            % ... but not win_u(S,X)

% FirstRed(State, Position)
fr(0,X) :-                     % X is first red in 0
	p(X),                  % ... if X is a position
	not o(0,X).            % ... that isn't even in the first overestimate (at 0)

fr(S1,X) :-                    % X is first red in S1 (=S+1)
	nxt(S,S1),
	o(S,X),                % ... if X was in the previous overestimate win_o(S,X)
	not o(S1,X),           % ... but isn't now in win_o(S1,X)
	not fin(S1).           % but exclude final state (we don't compute o(Final,...) )
%	not u(S1,X).           




% node(Color, State, Position)
node(g,S,X) :- fg(S,X).
node(r,S,X) :- fr(S,X).
node(y,S,X) :- y(X),fin(S).


outn(gr,S1,X,Y) :-
	m(X,Y), nxt(S,S1),
	g(X), fr(S,Y).		% GREEN --(s+1)--> FIRST-RED(s) 
outn(rg,S,X,Y) :- m(X,Y), r(X), fg(S,Y).

out(gg,X,Y) :- m(X,Y), g(X), g(Y).
out(gy,X,Y) :- m(X,Y), g(X), y(Y).
out(yg,X,Y) :- m(X,Y), y(X), g(Y).
% out(x,X,Y) :- m(X,Y), r(X), r(Y).
% out(x,X,Y) :- m(X,Y), r(X), y(Y).
% out(x,X,Y) :- m(X,Y), y(X), r(Y).
out(yy,X,Y) :- m(X,Y), y(X), y(Y).

Overwriting files/game_solve.dlv


In [3]:
!dlv files/move_graph.dlv files/game_solve.dlv -filter="node"

DLV [build BEN/Dec 17 2012   gcc 4.6.1]

{node(r,0,b), node(r,0,f), node(r,0,h), node(g,1,a), node(g,1,d), node(g,1,e), node(r,1,c), node(r,1,g), node(y,2,k), node(y,2,l), node(y,2,m), node(y,2,n)}


## Process DLV Output

For clear demonstratio, we parse the output of DLV into DataFrame

In [4]:
import pandas as pd
import subprocess
import re

def run_command(cmd):
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    output = result.stdout.decode()
    return output

def to_dataframe(output):
    output = output[1:-1]

    # Split the output into separate items
    items = output.split(", ")

    # Prepare the data for DataFrame
    data = []
    for item in items:
        # Extract the elements
        elements = re.findall(r'node\((.*?),(.*?),(.*?)\)', item)[0]
        data.append(elements)

    # Create DataFrame
    df = pd.DataFrame(data, columns=['color', 'state_id', 'node_label'])
    
    return df

if __name__ == "__main__":
    cmd = "dlv files/move_graph.dlv files/game_solve.dlv -filter='node',"
    output = run_command(cmd)
    state_df = to_dataframe(output)

In [5]:
state_df

,color,state_id,node_label
0,r,0,b
1,r,0,f
2,r,0,h
3,g,1,a
4,g,1,d
5,g,1,e
6,r,1,c
7,r,1,g
8,y,2,k
9,y,2,l


## Extract Move Graph Structure

In [6]:
import re

def read_edges_from_file(filename):
    edges = []
    with open(filename, 'r') as file:
        for line in file:
            # Extract edges that match the pattern "m(source, target)"
            match = re.match(r'm\(([^,]+),([^)]+)\)', line)
            if match:
                source = match.group(1)
                target = match.group(2)
                edges.append((source, target))
    return edges

if __name__ == "__main__":
    file_name = "files/move_graph.dlv"
    edges = read_edges_from_file(file_name)
    edge_df = pd.DataFrame(edges, columns=['source', 'target'])

In [7]:
edge_df

,source,target
0,a,b
1,a,c
2,k,l
3,c,d
4,c,e
5,l,e
6,l,m
7,d,e
8,e,d
9,e,m


## Color the Move Graph State by State

In [8]:
import pandas as pd
import pygraphviz as pgv
from IPython.display import Image

color_map = {'r': '#FFAAAA', 'g': '#AAFFAA', 'y': '#FFFFAA'}

# Create a directed graph from the first dataframe
G = pgv.AGraph(directed=True)

# Adding edges from the first dataframe
for _, row in edge_df.iterrows():
    G.add_edge(row['source'], row['target'])

# For each state, highlight the node and write to a separate dot file
for _, row in state_df.iterrows():
    # Find the node corresponding to the state
    node = G.get_node(row['node_label'])
    node.attr['fillcolor'] = color_map[row['color']]
    node.attr['style'] = 'filled'
    # Export to a Graphviz dot file
    G.write(f'output/state_{row["state_id"]}_graph.dot')
    
    with open(f'output/state_{row["state_id"]}_graph.dot', 'r') as f:
        lines = f.readlines()

    # Insert the new line before the last line
    lines.insert(-1, '{rank=same; "d" "e" "m"}\n')  # your custom script

    # Write the modified lines back to the file
    with open(f'output/state_{row["state_id"]}_graph.dot', 'w') as f:
        f.writelines(lines)

## dot to PNG

In [9]:
import os

# List all files in the output directory
files = os.listdir('output')

# Filter out the .dot files
dot_files = [file for file in files if file.endswith('.dot')]

# For each .dot file, convert it to a .png file using the 'dot' command
for dot_file in dot_files:
    png_file = os.path.splitext(dot_file)[0] + '.png'  # Replace .dot extension with .png
    command = f'dot -Tpng output/{dot_file} -o output/{png_file}'
    os.system(command)